In [1]:
import random
import math

## setup

In [2]:
def generate_vrptw_parameters(
    num_vehicles,
    num_customers,
    cost_range=(10, 50),
    time_range=(5, 30),
    demand_range=(5, 15),
    capacity=20,
    time_window_range=(0, 100),
    large_constant=1000,
    seed=None,
):
    """
    Generates random VRPTW parameters for a given number of vehicles and customers.

    Parameters:
    - num_vehicles (int): Number of vehicles.
    - num_customers (int): Number of customers (not including depots).
    - cost_range (tuple): Range for travel costs between nodes.
    - time_range (tuple): Range for travel times between nodes.
    - demand_range (tuple): Range for customer demands.
    - capacity (int): Capacity of each vehicle.
    - time_window_range (tuple): Range for time windows (start and end times).
    - large_constant (int): Large constant for constraints.

    Returns:
    - dict: A dictionary containing all sets and parameters.
    """
    if seed is not None:
        random.seed(seed)

    # Sets
    V = list(range(1, num_vehicles + 1))  # Vehicles labeled from 1 to num_vehicles
    N = list(
        range(num_customers + 2)
    )  # Nodes labeled from 0 to num_customers+1 (including depots)
    C = list(range(1, num_customers + 1))  # Customers labeled from 1 to num_customers

    # Parameters
    cij = {(i, j): random.randint(*cost_range) for i in N for j in N if i != j}
    tij = {(i, j): random.randint(*time_range) for i in N for j in N if i != j}
    di = {i: random.randint(*demand_range) for i in C}  # Customer demands
    q = capacity  # Vehicle capacity
    a = {i: random.randint(*time_window_range) for i in N}  # Start of time window
    b = {
        i: random.randint(a[i] + 1, time_window_range[1]) for i in N
    }  # End of time window, after start time
    K = large_constant  # Large constant for time window constraints

    # Ensuring depot time windows
    a[0], b[0] = 0, time_window_range[1]  # Starting depot time window
    a[num_customers + 1], b[num_customers + 1] = (
        0,
        time_window_range[1],
    )  # Returning depot time window

    return {
        "V": V,
        "N": N,
        "C": C,
        "cij": cij,
        "tij": tij,
        "di": di,
        "q": q,
        "a": a,
        "b": b,
        "K": K,
    }

In [3]:
n = 3  # num of customers
num_vehicles = 2
parameters = generate_vrptw_parameters(
    num_vehicles=num_vehicles, num_customers=n, seed=30
)
print(parameters)
V, N, C, cij, tij, di, q, a, b, K = parameters.values()

{'V': [1, 2], 'N': [0, 1, 2, 3, 4], 'C': [1, 2, 3], 'cij': {(0, 1): 44, (0, 2): 28, (0, 3): 49, (0, 4): 11, (1, 0): 49, (1, 2): 23, (1, 3): 26, (1, 4): 13, (2, 0): 35, (2, 1): 34, (2, 3): 18, (2, 4): 15, (3, 0): 39, (3, 1): 10, (3, 2): 43, (3, 4): 25, (4, 0): 11, (4, 1): 14, (4, 2): 20, (4, 3): 48}, 'tij': {(0, 1): 21, (0, 2): 17, (0, 3): 25, (0, 4): 16, (1, 0): 22, (1, 2): 7, (1, 3): 17, (1, 4): 5, (2, 0): 12, (2, 1): 30, (2, 3): 23, (2, 4): 26, (3, 0): 13, (3, 1): 29, (3, 2): 18, (3, 4): 23, (4, 0): 8, (4, 1): 26, (4, 2): 13, (4, 3): 25}, 'di': {1: 13, 2: 6, 3: 14}, 'q': 20, 'a': {0: 0, 1: 39, 2: 16, 3: 39, 4: 0}, 'b': {0: 100, 1: 72, 2: 27, 3: 48, 4: 100}, 'K': 1000}


In [4]:
print(f"a: {a} \nb: {b}")

a: {0: 0, 1: 39, 2: 16, 3: 39, 4: 0} 
b: {0: 100, 1: 72, 2: 27, 3: 48, 4: 100}


## SA

solutions for testing

In [5]:
exact_routes = [[0, 1, 4], [0, 2, 3, 4]]
infeasible_routes = [[0, 1, 4], [0, 2, 3]]
feasible_routes = [[0, 3, 4], [0, 2, 1, 4]]

check the feasibility of a solution

In [6]:
def is_feasible(routes, tij, di, q, a, b, K):
    """
    Checks if a solution (routes) is feasible based on VRPTW constraints.

    Parameters:
    - routes: List of routes, where each route is a list of customer visits for each vehicle.
    - cij: Cost matrix between nodes (i, j).
    - tij: Travel time matrix between nodes (i, j).
    - di: Demand for each customer.
    - q: Vehicle capacity.
    - a: Start time for time windows for each node.
    - b: End time for time windows for each node.
    - K: Large constant for time window constraints.

    Returns:
    - bool: True if the solution is feasible, False otherwise.
    """

    # Constraint (2.2): Each customer is visited exactly once
    customers_visited = set()
    for route in routes:
        for i in route:
            if i != 0 and i != len(b) - 1:  # Exclude depot nodes
                customers_visited.add(i)
    if len(customers_visited) != len(di):
        return False

    # Check each vehicle's route
    for k, route in enumerate(routes):
        # Initialize load and arrival time
        load = 0
        arrival_time = 0

        # Iterate through each customer in the route
        for idx in range(len(route) - 1):
            i, j = route[idx], route[idx + 1]

            # Constraint (2.3): Vehicle capacity
            if j in di:  # Skip depot nodes
                load += di[j]
                if load > q:
                    return False  # Exceeds vehicle capacity

            # Constraint (2.7): Time window and travel time constraints
            arrival_time = max(arrival_time + tij[i, j], a[j])
            if arrival_time > b[j]:
                return False  # Exceeds time window

        # Constraint (2.4) and (2.6): Route must start and end at depot
        if route[0] != 0 or route[-1] != len(b) - 1:
            return False

    # All constraints satisfied
    return True

sanity check on the previews solution

In [7]:
is_feasible(exact_routes, tij, di, q, a, b, K)

True

In [8]:
is_feasible(infeasible_routes, tij, di, q, a, b, K)

False

In [9]:
is_feasible(feasible_routes, tij, di, q, a, b, K)

True

Parameters for simulated annealing

In [10]:
initial_temperature = 1000
cooling_rate = 0.995
max_iterations = 1000

cost

In [11]:
def calculate_cost(x, cij, tij, a, b, q, di, routes):
    """
    Calculate the total cost of a given solution (routes).

    Parameters:
    - x: Decision variable dict to track arcs in the solution.
    - cij: Cost between nodes (i, j).
    - tij: Travel time between nodes (i, j).
    - a: Start time for time windows.
    - b: End time for time windows.
    - q: Vehicle capacity.
    - di: Customer demands.
    - routes: List of routes for each vehicle.

    Returns:
    - float: Total cost of the solution.
    """
    total_cost = 0

    x = {k: {} for k in range(num_vehicles)}  # Track arcs
    for k, route in enumerate(routes):
        load = 0
        arrival_time = 0

        for idx in range(len(route) - 1):
            i, j = route[idx], route[idx + 1]

            # Mark x[k, i, j] as 1 for the arc in the current route
            x[k, i, j] = 1

            # Add travel cost
            total_cost += cij[i, j]

            # Calculate arrival time at node j
            arrival_time = max(arrival_time + tij[i, j], a[j])

            # Check feasibility with time windows
            if arrival_time > b[j]:
                return float('inf')  # Penalize infeasible solution

            # Update load and check capacity
            if j in di:  # Skip depot nodes
                load += di[j]
                if load > q:
                    return float('inf')  # Penalize infeasible solution

    return total_cost


sanity check on exact solution

In [12]:
x = {k: {} for k in range(num_vehicles)}  # Track arcs
calculate_cost(x, cij, tij, a, b, q, di, exact_routes)

128

In [13]:
x = {k: {} for k in range(num_vehicles)}  # Track arcs
calculate_cost(x, cij, tij, a, b, q, di, infeasible_routes)

103

In [14]:
x = {k: {} for k in range(num_vehicles)}  # Track arcs
calculate_cost(x, cij, tij, a, b, q, di, feasible_routes)

149

Generate a new solution by randomly modifying the current routes.

In [15]:
def generate_new_solution(routes):
    """
    Generate a new solution by randomly modifying the current routes.

    Parameters:
    - routes: Current solution as a list of routes.

    Returns:
    - list: New solution with slightly modified routes.
    """
    new_routes = [route[:] for route in routes]  # Deep copy of current routes
    vehicle1, vehicle2 = random.sample(range(len(new_routes)), 2)
    if new_routes[vehicle1] and new_routes[vehicle2]:
        # Randomly swap nodes between two routes
        i = random.randint(1, len(new_routes[vehicle1]) - 2)
        j = random.randint(1, len(new_routes[vehicle2]) - 2)
        new_routes[vehicle1][i], new_routes[vehicle2][j] = (
            new_routes[vehicle2][j],
            new_routes[vehicle1][i],
        )
    return new_routes

In [16]:
def simulated_annealing(cij, tij, a, b, q, di, initial_routes, num_vehicles):
    """
    Solve the VRPTW using simulated annealing.

    Parameters:
    - cij: Cost between nodes (i, j).
    - tij: Travel time between nodes (i, j).
    - a: Start time for time windows.
    - b: End time for time windows.
    - q: Vehicle capacity.
    - di: Customer demands.
    - initial_routes: Initial feasible solution.
    - num_vehicles: Number of vehicles.

    Returns:
    - tuple: Best routes and their cost.
    """
    # Initialize variables
    current_routes = feasible_routes
    best_routes = current_routes
    x = {k: {} for k in range(num_vehicles)}  # Track arcs
    best_cost = calculate_cost(x, cij, tij, a, b, q, di, best_routes)

    temperature = 1000
    cooling_rate = 0.99

    while temperature > 1:
        # Generate a new solution
        new_routes = generate_new_solution(current_routes)

        # Calculate the cost of the new solution
        cost = calculate_cost(x, cij, tij, a, b, q, di, new_routes)

        # Accept or reject the new solution
        if cost < best_cost or random.uniform(0, 1) < math.exp((best_cost - cost) / temperature):
            current_routes = new_routes
            if cost < best_cost:
                best_cost = cost
                best_routes = new_routes

        # Cool down
        temperature *= cooling_rate

    return best_routes, best_cost

In [17]:
initial_routes = feasible_routes

best_routes, best_cost = simulated_annealing(
    cij=parameters["cij"],
    tij=parameters["tij"],
    a=parameters["a"],
    b=parameters["b"],
    q=parameters["q"],
    di=parameters["di"],
    initial_routes=initial_routes,
    num_vehicles=len(parameters["V"]),
)

print("Best Routes:")
for k, route in enumerate(best_routes):
    print(f"Vehicle {k+1} route:", route)

print("Best Cost:", best_cost)


Best Routes:
Vehicle 1 route: [0, 1, 4]
Vehicle 2 route: [0, 2, 3, 4]
Best Cost: 128
